In [1]:
import sys
sys.path.append('../')

In [2]:
import import_ipynb

In [3]:
from models.model_resnet18_FC             import Yolov1 as Yolo_resnet18
from models.model_resnet18_CNN_BLOCK_8_FC import Yolov1 as Yolo_resnet18_ft

from models.model_resnet50_FC             import Yolov1 as Yolo_resnet50
from models.model_resnet50_CNN_BLOCK_8_FC import Yolov1 as Yolo_resnet50_ft

from models.model_vgg19_FC                import Yolov1 as Yolo_vgg19
from models.model_vgg19_unfrozenCNN_FC    import Yolov1 as Yolo_vgg19_ft

importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\models\model_resnet18_FC.ipynb
importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\models\model_resnet18_CNN_BLOCK_8_FC.ipynb
importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\models\model_resnet50_FC.ipynb
importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\models\model_resnet50_CNN_BLOCK_8_FC.ipynb
importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\models\model_vgg19_FC.ipynb
importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\models\model_vgg19_unfrozenCNN_FC.ipynb


In [4]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader
from dataset.dataset import VOCDataset
from train.utils import (
    non_max_suppression,
    mean_average_precision,
    intersection_over_union,
    cellboxes_to_boxes,
    get_bboxes,
    plot_image,
    save_checkpoint,
    load_checkpoint,
)
from loss.loss import YoloLoss
from matplotlib import pyplot as plt

importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\dataset\dataset.ipynb
importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\train\utils.ipynb
importing Jupyter notebook from C:\Users\andri\Documents\Everything\YOLO-object-detection\train\..\loss\loss.ipynb


In [5]:
seed = 123
torch.manual_seed(seed)

DEVICE = "cuda" if torch.cuda.is_available else "cpu"

LEARNING_RATE = 2e-5                      # try different learning rates
# WEIGHT_DECAY = 0                        # try something here, why 0??
BATCH_SIZE = 64
EPOCHS = 100
PIN_MEMORY = True
SAVE_MODEL_PATH = "../output/models/best_model.pth"
IMG_DIR = "../dataset/VOC_dataset/VOC2012_train_val/VOC2012_train_val/JPEGImages/"
LABEL_TRAIN_DIR = "../dataset/labels/train/"
LABEL_VAL_DIR = "../dataset/labels/val/"

In [6]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes

        return img, bboxes

In [7]:
transform = Compose([transforms.Resize((224, 224)), transforms.ToTensor(),])

In [8]:
def train_fn(train_loader, model, optimizer, loss_fn):
    mean_loss = []

    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    mean_loss = sum(mean_loss)/len(mean_loss)
    return mean_loss

In [9]:
def val_fn(val_loader, model, optimizer, loss_fn):
    model.eval()
    mean_loss = []
    
    for batch_idx, (x, y) in enumerate(val_loader):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())

    mean_loss = sum(mean_loss)/len(mean_loss)
        
    model.train()
    return mean_loss

In [10]:
# Yolo parameters
split_size = 7 # split size
box_count = 2 # bounding boxes per split
class_count = 20 # number of classes

In [11]:
candidate_models = [Yolo_resnet18(S=split_size, B=box_count, C=class_count).to(DEVICE),
                    Yolo_vgg19(S=split_size, B=box_count, C=class_count).to(DEVICE)]

In [12]:
# model = Yolo_resnet18(S=split_size, B=box_count, C=class_count).to(DEVICE)
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# loss_fn = YoloLoss(S=split_size, B=box_count, C=class_count)

In [13]:
train_dataset = VOCDataset(
    transform=transform,
    img_dir=IMG_DIR,
    label_dir=LABEL_TRAIN_DIR,
    S=split_size, 
    B=box_count, 
    C=class_count
)

val_dataset = VOCDataset(
    transform=transform, 
    img_dir=IMG_DIR, 
    label_dir=LABEL_VAL_DIR,
    S=split_size, 
    B=box_count, 
    C=class_count
)

In [14]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=PIN_MEMORY,
    shuffle=True,
    drop_last=True,
)

In [15]:
# # early stopping variables
# EARLY_STOPPING_LIMIT = 100
# epochs_since_last_improvement = 0
# best_loss = float('inf')
# best_model = model.state_dict()

In [ ]:
model_mean_loss_history_train = {}
model_mean_avg_prec_history_train = {}
model_mean_loss_history_val = {}
model_mean_avg_prec_history_val = {}

for model in candidate_models:  
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fn = YoloLoss(S=split_size, B=box_count, C=class_count)

    mean_loss_history_train = []
    mean_avg_prec_history_train = []
    mean_loss_history_val = []
    mean_avg_prec_history_val = []
    for epoch in tqdm(range(EPOCHS)):
        pred_boxes, target_boxes = get_bboxes(train_loader, model, iou_threshold=0.5, threshold=0.4)
        mean_avg_prec_train = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5)
        # print(f"Train mAP: {mean_avg_prec}")
        mean_loss_train = train_fn(train_loader, model, optimizer, loss_fn)
        #print(f"Mean loss was {mean_loss}")

        # if EPOCHS % 10:
        # validation
        with torch.no_grad():
            pred_boxes, target_boxes = get_bboxes(val_loader, model, iou_threshold=0.5, threshold=0.4)
            mean_avg_prec_val = mean_average_precision(pred_boxes, target_boxes, iou_threshold=0.5)
            mean_loss_val = val_fn(val_loader, model, optimizer, loss_fn)

        # append metrics
        mean_avg_prec_history_train.append(mean_avg_prec_train)
        mean_loss_history_train.append(mean_loss_train)
        mean_avg_prec_history_val.append(mean_avg_prec_val)
        mean_loss_history_val.append(mean_loss_val)
        
    model_mean_loss_history_train[str(model)]     = mean_loss_history_train
    model_mean_avg_prec_history_train[str(model)] = mean_avg_prec_history_train
    model_mean_loss_history_val[str(model)]       = mean_loss_history_val
    model_mean_avg_prec_history_val[str(model)]   = mean_avg_prec_history_val

#     # early stopping
#     if mean_loss_val < best_loss:
#         epochs_since_last_improvement = 0
#         best_loss = mean_loss_val
#         best_model = model.state_dict() 
#     else:
#         epochs_since_last_improvement += 1
#         if epochs_since_last_improvement >= EARLY_STOPPING_LIMIT:
#             print(f"Early stopped at epoch {epoch}!")
#             print(f"Best model at epoch {epoch-epochs_since_last_improvement}!")
#             model.load_state_dict(best_model)
#             break

  1%|▊                                                                             | 1/100 [06:21<10:29:30, 381.53s/it]

In [ ]:
fig = plt.figure(figsize=(20, 10))
fig, ax = plt.subplots(2, 1)
for model in candidate_models:
    
    ax[0].plot(range(EPOCHS), model_mean_avg_prec_history_train[str(model)], label = f'train {str(model)}')
    ax[0].plot(range(EPOCHS), model_mean_avg_prec_history_val[str(model)], label = f'val {str(model)}')
    #ax[0, 0].title("Mean trainining average precision")
    #plt.legend()
    #plt.show()
    
    ax[1].plot(range(EPOCHS), model_mean_loss_history_train[str(model)], label = f'train {str(model)}')
    ax[1].plot(range(EPOCHS), model_mean_loss_history_val[str(model)], label = f'val {str(model)}')
    #plt.title("Mean training loss")
    #plt.legend()
    #plt.show()

ax[0].legend()
ax[1].legend()
plt.show()   

In [ ]:
# pick best model
best_model = None
best_precision = 0
for model in candidate_models:
    model_precision = model_mean_avg_prec_history_val[str(model)][-1]
    if model_precision > best_precision:
        best_model = model
        best_precision = model_precision

In [ ]:
print(f'Best model: {best_model}')
print(f'Precision: {round(best_precision.item(), 2)}')

In [ ]:
best_model.eval()

In [ ]:
# Plot predictions on train data
with torch.no_grad():
    for x, y in train_loader:
        x = x.to(DEVICE)
        out = best_model(x)
        #classes, confidences = get_class_confidences(out)
        #break
        bboxes = cellboxes_to_boxes(out)
        for idx in range(BATCH_SIZE):
            bboxes_nms = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4)
            plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes_nms)
        break

In [ ]:
# Plot predictions on validation data
with torch.no_grad():
    for x, y in val_loader:
        x = x.to(DEVICE)
        out = best_model(x)
        bboxes = cellboxes_to_boxes(out)
        for idx in range(BATCH_SIZE):
            bboxes_nms = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4)
            plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes_nms)
        break

In [ ]:
# saving a model
torch.save(best_model.state_dict(), SAVE_MODEL_PATH)